In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [ ]:
df = pd.read_csv('/kaggle/input/tabular-playground-series-jun-2022/data.csv')
ind = df.isna()

In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
def LinearRegressionPredictor(column):
    print('Creating model for', column)
    df_f10 = df[~df[column].isna()]
    X = df_f10.drop([column, 'row_id'], axis=1)
    y = df_f10[column]
    #Replace NAs with 0
    imp = SimpleImputer(strategy="constant", fill_value=0)
    X[:] = imp.fit_transform(X)
    X_train, X_test, y_train, y_test = train_test_split(X, y)
    #Linear Regression (need SGD because dataset is too large)
    coefs = np.zeros(X_train.shape[1])
    intes = 0
    lr = 1e-5
    for i in range(1000):
        inds = np.random.choice(X_train.shape[0], size=256, replace=False)
        X_inp = X_train.iloc[inds,:].values
        y_inp = y_train.iloc[inds].values
        error = y_inp - np.matmul(X_inp, coefs) - intes
        coefs += lr * 2 * np.matmul(error, X_inp)
        intes += lr * 2 * np.sum(error)
    test_error = y_test - np.matmul(X_test, coefs) - intes
    print('Test MSE:', np.mean(np.square(test_error)))
    return coefs, intes

In [ ]:
models = []
nums = [15, 25, 25, 15]
for i in [1, 3, 4]:
    for j in range(nums[i-1]):
        string = 'F_'+str(i)+'_'+str(j)
        models.append(LinearRegressionPredictor(string))

In [ ]:
#Actually there are more than one NA value in some rows, but we don't care for now.
df.isna().sum(axis=1).value_counts()

In [ ]:
preds = []
for i in range(15+25+15):
    j = i
    if (j >= 15): 
        j += 25
    arr = df[df[df.columns[j+1]].isna()]
    arr = arr.drop(['row_id', df.columns[j+1]], axis=1).fillna(value=0)
    preds.append(np.matmul(arr, models[i][0]) + models[i][1])

In [ ]:
sub = pd.read_csv('/kaggle/input/tabular-playground-series-jun-2022/sample_submission.csv')
missing_cols = np.argwhere(df.isna().values)
counts = []
for i in range(15+25+25+15):
    counts.append(0)
result = []
for i in range(missing_cols.shape[0]):
    ind = missing_cols[i][1] - 1
    if ind > 15:
        ind -= 25
    result.append(preds[ind].values[counts[ind]])
    counts[ind] += 1
sub['value'] = np.array(result)

In [ ]:
sub.head()

In [ ]:
sub.to_csv('submission.csv', index=False)